In [1]:
%pip install pyarrow
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Paul\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Paul\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

c:\Users\Paul\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the dataset

In [27]:
# Load the dataset
dataset = load_dataset("jonghyunlee/UniProt_function_text_descriptions")

# Diviser les données en train, validation et test
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
test_valid = dataset["test"].train_test_split(test_size=0.5, seed=42)

dataset = DatasetDict({
    'train': dataset['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']
})

In [29]:
# display % of training data with label=1
np.array(dataset['train']['protein_name']).sum()/len(dataset['train']['protein_name'])

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U1263'), dtype('<U1263')) -> None

Model

In [7]:
# Définir le point de contrôle du modèle
model_checkpoint = "dmis-lab/biobert-base-cased-v1.2"

# Extraire les séquences et les noms de protéines
sequences = dataset['train']['sequence']
protein_names = dataset['train']['protein_name']

# Créer une liste unique des noms de protéines (labels)
label_names = list(set(protein_names))
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {label: i for i, label in enumerate(label_names)}

# Ajouter les labels au jeu de données
dataset = dataset.map(lambda x: {'labels': label2id[x['protein_name']]})

# Charger le modèle
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=len(label_names), id2label=id2label, label2id=label2id
)

Map: 100%|██████████| 464395/464395 [00:23<00:00, 19495.50 examples/s]
c:\Users\Paul\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Paul\.cache\huggingface\hub\models--dmis-lab--biobert-base-cased-v1.2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weig

In [8]:
# Display architecture
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

Preprocess data

In [10]:
# Define the tokenizer
#tokenizer = AutoTokenizer.from_pretrained("dotan1111/BioTokenizer-BFD-BPE-100")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
# Create tokenize function
def tokenize_function(examples):
    # Extract text
    text = examples["sequence"]
    
    # Tokenize and truncate text
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )
    
    return tokenized_inputs

In [12]:
# Tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
print(tokenized_dataset)

Map: 100%|██████████| 464395/464395 [00:28<00:00, 16337.61 examples/s]

DatasetDict({
    train: Dataset({
        features: ['entry', 'entry_name', 'protein_name', 'sequence', 'function', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 464395
    })
})


In [13]:
# Create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Evaluation

In [14]:
# Import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [15]:
# Define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    
    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

Apply untrained model to text

In [16]:
# Define list of examples
text_list = ["MVLSPADKTNVKAAW", "GAGGAGAAGGTGTGGCG", "MGDVEKGKKIFIMKCSQCHTVEKGGKHKTGPNEKAGA"]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # Tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # Compute logits
    logits = model(inputs).logits
    # Convert logits to label
    predictions = torch.argmax(logits)
    
    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
MVLSPADKTNVKAAW - Ribosomal protein S6 kinase beta-2 (S6K-beta-2) (S6K2) (EC 2.7.11.1) (70 kDa ribosomal protein S6 kinase 2) (p70 ribosomal S6 kinase beta) (p70 S6 kinase beta) (p70 S6K-beta) (p70 S6KB)
GAGGAGAAGGTGTGGCG - Magnetosome protein MamA (MM24.3)
MGDVEKGKKIFIMKCSQCHTVEKGGKHKTGPNEKAGA - Ribosomal protein S6 kinase beta-2 (S6K-beta-2) (S6K2) (EC 2.7.11.1) (70 kDa ribosomal protein S6 kinase 2) (p70 ribosomal S6 kinase beta) (p70 S6 kinase beta) (p70 S6K-beta) (p70 S6KB)


Train model

In [17]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                         r=4,
                         lora_alpha=32,
                         lora_dropout=0.01,
                         target_modules=['q_lin'])

In [18]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [22]:
# Exemple de configuration PEFT ajustée avec LoraConfig
peft_config = LoraConfig(
    task_type="SEQUENCE_CLASSIFICATION",
    target_modules=[
        "bert.encoder.layer.0.attention.self.query",
        "bert.encoder.layer.0.attention.self.key",
        "bert.encoder.layer.0.attention.self.value",
        "bert.encoder.layer.0.output.dense",
        "bert.encoder.layer.1.attention.self.query",
        "bert.encoder.layer.1.attention.self.key",
        "bert.encoder.layer.1.attention.self.value",
        "bert.encoder.layer.1.output.dense",
        "bert.encoder.layer.2.attention.self.query",
        "bert.encoder.layer.2.attention.self.key",
        "bert.encoder.layer.2.attention.self.value",
        "bert.encoder.layer.2.output.dense",
        "bert.encoder.layer.3.attention.self.query",
        "bert.encoder.layer.3.attention.self.key",
        "bert.encoder.layer.3.attention.self.value",
        "bert.encoder.layer.3.output.dense",
        # Continuez pour les autres couches si nécessaire
    ],
    r=8,  # Configuration spécifique à LoRA
    lora_alpha=32,
    lora_dropout=0.1,
)

In [23]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 270,336 || all params: 198,411,343 || trainable%: 0.1363


In [20]:
#pour voir les couches du modele
for name, module in model.named_modules():
    print(name)



bert
bert.embeddings
bert.embeddings.word_embeddings
bert.embeddings.position_embeddings
bert.embeddings.token_type_embeddings
bert.embeddings.LayerNorm
bert.embeddings.dropout
bert.encoder
bert.encoder.layer
bert.encoder.layer.0
bert.encoder.layer.0.attention
bert.encoder.layer.0.attention.self
bert.encoder.layer.0.attention.self.query
bert.encoder.layer.0.attention.self.key
bert.encoder.layer.0.attention.self.value
bert.encoder.layer.0.attention.self.dropout
bert.encoder.layer.0.attention.output
bert.encoder.layer.0.attention.output.dense
bert.encoder.layer.0.attention.output.LayerNorm
bert.encoder.layer.0.attention.output.dropout
bert.encoder.layer.0.intermediate
bert.encoder.layer.0.intermediate.dense
bert.encoder.layer.0.intermediate.intermediate_act_fn
bert.encoder.layer.0.output
bert.encoder.layer.0.output.dense
bert.encoder.layer.0.output.LayerNorm
bert.encoder.layer.0.output.dropout
bert.encoder.layer.1
bert.encoder.layer.1.attention
bert.encoder.layer.1.attention.self
bert.e

In [24]:
# Hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10

In [25]:
# Define training arguments
training_args = TrainingArguments(
    output_dir=model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

c:\Users\Paul\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [30]:
# Create trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()

KeyError: 'validation'

Generate prediction

In [ ]:
model.to('cpu') # moving to cpu

print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu")
    
    logits = model(inputs).logits
    predictions = torch.argmax(logits, dim=1)
    
    print(text + " - " + id2label[predictions.tolist()[0]])